In [1]:
import pandas as pd
import numpy as np 
data_set= pd.read_csv('/home/boras/Downloads/monatszahlen2112_verkehrsunfaelle.csv')
data_set.columns

Index(['MONATSZAHL', 'AUSPRAEGUNG', 'JAHR', 'MONAT', 'WERT', 'VORJAHRESWERT',
       'VERAEND_VORMONAT_PROZENT', 'VERAEND_VORJAHRESMONAT_PROZENT',
       'ZWOELF_MONATE_MITTELWERT'],
      dtype='object')

In [3]:
## we notice that we need to delete the useless features

In [4]:
infos= data_set.copy()
infos = infos.drop(['VORJAHRESWERT','VERAEND_VORMONAT_PROZENT','VERAEND_VORJAHRESMONAT_PROZENT','ZWOELF_MONATE_MITTELWERT','VORJAHRESWERT'], axis=1)

In [5]:
infos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1995 entries, 0 to 1994
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MONATSZAHL   1995 non-null   object 
 1   AUSPRAEGUNG  1995 non-null   object 
 2   JAHR         1995 non-null   int64  
 3   MONAT        1995 non-null   object 
 4   WERT         1911 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 78.1+ KB


In [6]:
## we deduce that there is additional rows for features than our label
## + 3 features are categorical and 2 are numerical

In [7]:
## for the numbers
infos.describe()

,JAHR,WERT
count,1995.000000,1911.000000
mean,2010.463158,1440.540031
std,6.326381,4655.514611
min,2000.000000,0.000000
25%,2005.000000,41.000000
50%,2010.000000,399.000000
75%,2016.000000,864.000000
max,2021.000000,46988.000000


In [8]:
#describe as categorical
infos.astype('object').describe()

,MONATSZAHL,AUSPRAEGUNG,JAHR,MONAT,WERT
count,1995,1995,1995,1995,1911.0
unique,3,3,22,265,930.0
top,Verkehrsunfälle,insgesamt,2010,Summe,21.0
freq,855,855,91,147,21.0
